In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_data = np.loadtxt('train.csv', delimiter=",", skiprows=1)
test_data = np.loadtxt('test.csv', delimiter=",", skiprows=1)
x_train=train_data[:,1:]

In [ ]:
x_train=train_data[:,1:]
x_test=test_data[:,1:]
y_train=train_data[:,0]
y_test=test_data[:,0]
#Lets check that all the classes are exist in our data
unique_values = np.unique(y_train)
number_of_unique_values = len(unique_values)
print(number_of_unique_values)

In [ ]:
def plot_image(X, title, ax):
    ax.imshow(X, cmap='gray')
    ax.axis('off') # hide the axes ticks
    ax.set_title(title, color= 'black', fontsize=25)

def printExamples(x_train,y_train,numOfclass,numOfExamples=4):
    fig, axes = plt.subplots(numOfclass,numOfExamples, figsize=(12,12))
    for i in range(0,numOfclass):
        idx = np.random.choice(np.where(y_train == i)[0],size=numOfExamples)
        for j in range(numOfExamples):
            ax = axes[i, j]
            img = x_train[idx[j],:].reshape(28,28) 
            title = str(int(y_train[idx[j]])) 
            plot_image(img, title,ax)   
    plt.tight_layout()
    plt.show()

printExamples(x_train,y_train,number_of_unique_values)

In [ ]:
def train_val_split(x_train,y_train,val_pct=0.2):
    X_train=x_train[:round((1-val_pct)*x_train.shape[0]),1:]
    X_val=x_train[round((1-val_pct)*x_train.shape[0]):,1:]
    Y_train=y_train[:round((1-val_pct)*y_train.shape[0])]
    Y_val=y_train[round((1-val_pct)*y_train.shape[0]):]

    return X_train,X_val,Y_train,Y_val

In [ ]:
def normalize(X,type='minMax'):
    eps = 1e-8
    if type!='minMax' and type!= 'Zscore':
        print("Invalid type of normalization.Using MinMax as default type\n")
        type ='minMax'
    if type=='minMax':
        # get min and max per feature (except the first one which corresponds to the bias term)
        X_min = np.min(X[:, 1:], axis=0)
        X_max = np.max(X[:, 1:], axis=0)
        return (X[:, 1:] - X_min)/(X_max - X_min + eps)
    else:
        X_mean=np.mean(X[:, 1:], axis=0)
        X_sd=np.std(X[:, 1:], axis=0)
        return (X[:, 1:] - X_mean)/(X_sd)

In [ ]:
def softmax(z):
    # Shift z by subtracting its max value in each row for numerical stability
    z_shifted = z - np.max(z, axis=1, keepdims=True)
    
    # Calculate the exponential of the shifted z
    z_exp = np.exp(z_shifted)
    
    # Normalize the exponentials by the sum across classes (columns) for each sample (row)
    softmax_output = z_exp / np.sum(z_exp, axis=1, keepdims=True)
    # z_k = z - np.ones((z.shape[0], 1)) @ (np.max(z, axis=0).reshape(1, z.shape[1]))
    # return np.exp(z_k) / np.sum(np.exp(z_k), axis=0).reshape(1, z_k.shape[1])
    return softmax_output

In [ ]:
def oneHot(Y,numOfClass):
    y_oneHot=np.zeros((Y.size,numOfClass))
    y_oneHot[np.arange(Y.size), Y.astype(int)]= 1
    
    return y_oneHot

In [ ]:
def croosEntropy(softmax_ypred,y_oneHot_true):
    eps = 1e-8
    n=softmax_ypred.shape[0]
    #return np.mean(-(np.log(softmax_ypred[(y_oneHot_true==1)]+eps)))
    L=np.sum(np.log(softmax_ypred +eps )*y_oneHot_true)
    return -L/n

In [ ]:
def dL_dw(softmax_z, y_true, x,reg_coeff,w):
    n = x.shape[0]  # number of samples
    dL = (1 / n) * x.T @ (softmax_z-y_true)+2*reg_coeff*w
    return dL
  

In [ ]:
def test(X,W,y=None):
    N_test = X.shape[0]  # number of training samples
    # matrix-vector multiplication
    z = X @ W
   

    softmax_z = softmax(z)
    prediction=np.argmax(softmax_z, axis=1)
    if y is None:
        return prediction
    # calculate loss
    test_loss = croosEntropy(softmax_z,y)
    # calc. accuracy
    labels_indices = np.argmax(y, axis=1)
    prediction = np.array(prediction).reshape(-1)
    accuracy = np.mean(labels_indices==prediction)
    return accuracy, test_loss

In [ ]:

def logistic_classification(X_train,X_val,y_train,y_val,yhot_test,batch_size,eta,epochs,reg_coeff):
    N = X_train.shape[0]  # number of training samples
    d = X_train.shape[1]  # dimension = 784 + 1 (for the bias term)
    W = np.random.normal(loc=0.0, scale=0.01, size=(d, 10))  # initalize weights iid from a gaussian with small noise
    train_losses,train_accuracy, val_losses, val_accuracy = ([] for i in range(4))

    # iterations over entire dataset
    for epoch in range(epochs):
        loss = 0
        train_acc=0
        print(epoch)
        # batch iterations whithin each dataset iteration 
        for batch_idx, idx_start in enumerate(range(0, N, batch_size)):
            idx_end = min(idx_start + batch_size, N)
            X_batch = X_train[idx_start:idx_end, :]  # take all data in the current batch
            y_batch = y_train[idx_start:idx_end,]  # take relevant labels

            # matrix-vector multiplication
            z = X_batch @ W  
            # calc. probaility of y_j = 1 for each input (M,)
            softmax_z = softmax(z)
            pred=np.argmax(softmax_z, axis=1)
            # calc. accuracy
            labels_in = np.argmax(y_batch, axis=1)
            prediction = np.array(pred).reshape(-1)
            train_acc += np.mean(labels_in==pred)
            # calculate loss
            batch_loss = croosEntropy(softmax_z,y_batch)+ reg_coeff*(np.linalg.norm(W)**2)
            loss += batch_loss
            
            # compute gradient of the loss w.r.t W
            delta_W = dL_dw(softmax_z, y_batch, X_batch,reg_coeff,W)
            
            # update W
            W = W - eta * delta_W
        ##### validation ####
        val_acc,val_loss = test(X_val,W, y_val)
        
        # save for plotting
        train_losses.append(loss / batch_idx)
        train_accuracy.append(train_acc/batch_idx)
        val_losses.append(val_loss)
        val_accuracy.append(val_acc)

   
    return train_losses,train_accuracy, val_losses, val_accuracy,W

In [ ]:

x_train,x_val,y_train,y_val=train_val_split(x_train,y_train,0.2)


In [ ]:
x_train[:, 1:] = normalize(x_train)
x_val[:, 1:] = normalize(x_val)
x_test[:, 1:] = normalize(x_test)
# add constant "1" for the bias term
N_train = x_train.shape[0]  # number of training samples
x_train = np.concatenate((np.ones((N_train, 1)), x_train.reshape(N_train, -1)), axis=1)
N_test = x_test.shape[0]  # number of test samples
x_test = np.concatenate((np.ones((N_test, 1)),x_test.reshape(N_test, -1)), axis=1)


In [ ]:
N_val = x_val.shape[0]  # number of training samples
x_val = np.concatenate((np.ones((N_val, 1)), x_val.reshape(N_val, -1)), axis=1)
yhot_train=oneHot(y_train,len(unique_values))
yhot_val=oneHot(y_val,len(unique_values))
yhot_test=oneHot(y_test,len(unique_values))

In [ ]:
print(x_train.shape)
print(x_train[0:20,0])

In [ ]:
batch_size = 64  # batch size
eta = 0.005 # learning rate
epochs = 40  # number of times to iterate over the entier dataset
reg_coeff=0.005
train_losses,train_accuracy, val_losses, val_accuracy,W=logistic_classification(x_train,x_val,yhot_train,yhot_val,yhot_test,batch_size,eta,
                                                                                epochs,reg_coeff)

In [ ]:
# plot loss and accuracy on validation set
steps = np.arange(epochs)

fig, ax1 = plt.subplots()

ax1.set_xlabel('epochs')
ax1.set_ylabel('loss')
#ax1.set_title('test loss: %.3f, test accuracy: %.3f' % (test_loss, test_acc))
ax1.plot(steps, train_losses, label="train loss", color='red')
ax1.plot(steps, val_losses, label="val loss", color='green')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('acccuracy')  # we already handled the x-label with ax1
ax2.plot(steps, val_accuracy, label="val acc", color='blue')
ax2.plot(steps, train_accuracy, label="train acc", color='yellow')

fig.legend()
fig.tight_layout()
plt.show()

In [ ]:
print(val_accuracy[-1])
print(train_accuracy[-1])    

Part 3

In [ ]:
def Relu(x):
    return np.maximum(x,0)

In [ ]:
def Relu_derive(x):
    res=np.zeros((x.shape[0],x.shape[1]))
    res[x>0]=1
    return res

In [ ]:
def test_NN(x,w1,w2,b1,b2,y=None):
    N_test = x.shape[0]  # number of training samples
    # matrix-vector multiplication
    z1=(x @ w1) +b1
    h=Relu(z1)
    y_pred=softmax((h @ w2) +b2)
    prediction=np.argmax(y_pred, axis=1)
    if y is None:
        return prediction
    # calculate loss
    test_loss = croosEntropy(y_pred,y)+ reg_coeff*(np.linalg.norm(w1)**2 + np.linalg.norm(w2)**2)
    # calc. accuracy
    labels_indices = np.argmax(y, axis=1)
    prediction = np.array(prediction).reshape(-1)
    accuracy = np.mean(labels_indices==prediction)
    return accuracy, test_loss

In [ ]:
def NN_classification(x_train,x_val,y_train,y_val,batch_size,layer_size,eta, epochs,reg_coeff):
    N = x_train.shape[0]
    num_of_pixels=x_train.shape[1]
    #unique_values = np.unique(y_train)
    num_of_class = 10
    w1=np.random.rand(num_of_pixels,layer_size)
    w2=np.random.rand(layer_size,num_of_class)
    b1=np.random.rand(1,layer_size)
    b2=np.random.rand(1,num_of_class)

    train_losses,train_accuracy, val_losses, val_accuracy = ([] for i in range(4))
    # iterations over entire dataset
    for epoch in range(epochs):
        loss = 0
        train_acc=0
        print(epoch)
        # batch iterations whithin each dataset iteration 
        for batch_idx, idx_start in enumerate(range(0, N, batch_size)):
            idx_end = min(idx_start + batch_size, N)
            x_batch = x_train[idx_start:idx_end, :]  # take all data in the current batch
            y_batch = y_train[idx_start:idx_end,]  # take relevant labels
            z1=(x_batch @ w1) +b1
            h=Relu(z1)
            y_pred=softmax((h @ w2) +b2)
            pred=np.argmax(y_pred, axis=1)
            # calc. accuracy
            labels_in = np.argmax(y_batch, axis=1)
            prediction = np.array(pred).reshape(-1)
            train_acc += np.mean(labels_in==pred)
            # calculate loss
            batch_loss = croosEntropy(y_pred,y_batch)+ reg_coeff*(np.linalg.norm(w1)**2 + np.linalg.norm(w2)**2)
            loss += batch_loss
            
            # Compute gradients
            dLdz2 = y_pred-y_batch
            batch_len = len(range(idx_start,idx_end))         
            dLdb2 = (1. / batch_len) * np.ones((1,batch_len)) @ dLdz2
            dLdw2 = (1. / batch_len) * h.T @dLdz2 

            dLdz1 = (dLdz2 @ w2.T) * Relu_derive(z1) #dLdh=w2
            
            dLdw1 = (1. / batch_len) * x_batch.T @ dLdz1
            dLdb1 = (1. / batch_len) * np.ones((1,batch_len)) @ dLdz1 
            
            w2 = w2 * (1-eta*reg_coeff) - eta * (dLdw2 + 2*w2*reg_coeff)
            b2 = b2 - eta * dLdb2
            w1 = w1 * (1-eta*reg_coeff) - eta * (dLdw1 + 2*reg_coeff * w1)
            b1 = b1 - eta * dLdb1
            
        ##### validation ####
        val_acc,val_loss = test_NN(x_val,w1,w2,b1,b2, y_val)
        
        # save for plotting
        train_losses.append(loss / batch_idx)
        train_accuracy.append(train_acc/batch_idx)
        val_losses.append(val_loss)
        val_accuracy.append(val_acc)

   
    return train_losses,train_accuracy, val_losses, val_accuracy





In [ ]:
batch_size = 64  # batch size
eta = 0.005 # learning rate
epochs = 40  # number of times to iterate over the entier dataset
reg_coeff=0.005
layer_size=50
train_losses,train_accuracy, val_losses, val_accuracy=NN_classification(x_train,x_val,yhot_train,yhot_val,batch_size,layer_size,eta,
                                                                         epochs,reg_coeff)

In [ ]:
# plot loss and accuracy on validation set
steps = np.arange(epochs)

fig, ax1 = plt.subplots()

ax1.set_xlabel('epochs')
ax1.set_ylabel('loss')
ax1.plot(steps, train_losses, label="train loss", color='red')
ax1.plot(steps, val_losses, label="val loss", color='green')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('acccuracy')  # we already handled the x-label with ax1
ax2.plot(steps, val_accuracy, label="val acc", color='blue')
ax2.plot(steps, train_accuracy, label="train acc", color='yellow')

fig.legend()
fig.tight_layout()
plt.show()

In [218]:
print(val_accuracy[-1])
print(train_accuracy[-1])    

0.8300892857142858
0.8347415951359084
